In [1]:
from functions.chat_bot_v2 import *
from functions.filomena_utils_v2 import *

2024-01-05 23:38:34.478 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`


In [2]:
fil = Filomena()

In [3]:
import os 
files = [f'text_data/{file}' for file in os.listdir('text_data')]

In [4]:
fil.initialize(files= files)

In [5]:
fil.generate_response('Hello')

"Hey there, Fellow Foodie! I'm Filomena, FlavourFlix' virtual assistant. How can I assist you today?"

In [6]:
fil.generate_response('Tell me about SBO restaurant in évora')

"The restaurant 'DOP' is a Portuguese restaurant located in Porto. It has a modern style and the average price is 70. The restaurant is closed on Mondays and Sundays, and open for lunch and dinner from Tuesday to Saturday."

In [7]:
fil.generate_response('Tell me about Portugalia')

"The restaurant 'Portugália - Cais do Sodré' is a seafood restaurant located in Lisboa. The average price is 20 units. The restaurant is open from Monday to Sunday, from 12:00 to 22:30."

In [7]:
fil.generate_response('Tell me about Galina da vizinha')